In [86]:
#Lib
import pandas as pd
from joblib import load
# agglomerative clustering
from numpy import unique
from numpy import where
from sklearn.cluster import KMeans
#Zips
import py7zr
#elbow point
%pip install kneed
from kneed import KneeLocator
# plots 
%pip install plotly
import plotly.express as px
import matplotlib.pyplot as plt

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


In [87]:
with py7zr.SevenZipFile('../../../../data/WA_Fn-UseC_-Telco-Customer-Churn/original_train.7z', mode='r') as z:
    z.extractall(path='original_train/')
with py7zr.SevenZipFile('../../../../data/WA_Fn-UseC_-Telco-Customer-Churn/test/x_test.7z', mode='r') as z:
    z.extractall(path='x_test/')
with py7zr.SevenZipFile('../../../../data/WA_Fn-UseC_-Telco-Customer-Churn/test/y_test.7z', mode='r') as z:
    z.extractall(path='y_test/')
with py7zr.SevenZipFile('../../../../data/WA_Fn-UseC_-Telco-Customer-Churn/test/y_test.7z', mode='r') as z:
    z.extractall(path='y_test/')
with py7zr.SevenZipFile('../../../../data/WA_Fn-UseC_-Telco-Customer-Churn/WA_Fn-UseC_-Telco-Customer-Churn.7z', mode='r') as z:
    z.extractall(path='WA_Fn-UseC_-Telco-Customer-Churn/')

In [88]:
target = 'Churn'

In [89]:
train = pd.read_csv('./original_train/original_train.csv')

x_test = pd.read_csv('./x_test/x_test.csv')
y_test = pd.read_csv('./y_test/y_test.csv')

x_train = train.drop([target], axis=1)
y_train = pd.DataFrame(train[target])
original_data = pd.read_csv('./WA_Fn-UseC_-Telco-Customer-Churn/WA_Fn-UseC_-Telco-Customer-Churn.csv')

In [90]:
x_test.dtypes

tenure            float64
StreamingTV       float64
PaymentMethod     float64
MonthlyCharges    float64
TotalCharges      float64
dtype: object

In [91]:
model = load('../../joblibs/WA_Fn-UseC_-Telco-Customer-Churn/model/classification-model.joblib')
model

/opt/anaconda3/lib/python3.9/site-packages/sklearn/base.py:329: UserWarning:

Trying to unpickle estimator LabelBinarizer from version 0.24.2 when using version 1.1.3. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations

/opt/anaconda3/lib/python3.9/site-packages/sklearn/base.py:329: UserWarning:

Trying to unpickle estimator MLPClassifier from version 0.24.2 when using version 1.1.3. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations



MLPClassifier(alpha=1e-05, hidden_layer_sizes=(5, 2), random_state=1,
              solver='lbfgs')

In [92]:
y_pred_prob = model.predict_proba(x_test)
y_pred = model.predict(x_test)
y_pred

/opt/anaconda3/lib/python3.9/site-packages/sklearn/base.py:443: UserWarning:

X has feature names, but MLPClassifier was fitted without feature names

/opt/anaconda3/lib/python3.9/site-packages/sklearn/base.py:443: UserWarning:

X has feature names, but MLPClassifier was fitted without feature names



array(['No', 'No', 'No', ..., 'No', 'No', 'No'], dtype='<U3')

In [93]:
x_ones = x_train
x_ones[target] = y_train
x_ones = x_ones.loc[x_ones[target] == 'Yes']
x_ones = x_ones.drop([target], axis=1)
#PREPAID_LINES
x_ones

,tenure,StreamingTV,PaymentMethod,MonthlyCharges,TotalCharges
0,0.880672,0.238870,1.466061,1.220945,0.342103
2,-0.463004,0.238870,-1.145117,-0.292844,0.285644
3,0.107040,0.238870,0.595668,1.249194,1.539877
12,1.613587,0.238870,1.466061,0.920182,0.587990
14,-1.195919,-1.071381,0.595668,0.159133,-1.327393
...,...,...,...,...,...
4916,-1.195919,0.238870,1.466061,-0.982441,0.299891
4927,-0.870179,-1.071381,-1.145117,0.052785,-1.040349
4928,-1.114484,1.549122,1.466061,-1.492576,0.827017
4929,-1.236636,-1.071381,-1.145117,0.695855,-0.269975


In [94]:
#Elbow point visualization graph
from sklearn.cluster import KMeans
from sklearn.preprocessing import MinMaxScaler
import plotly.graph_objects as go
import numpy as np
scaler = MinMaxScaler()
scaler.fit(x_ones)
X=scaler.transform(x_ones)
inertia = []
max_clusters = x_ones.shape[1] + 1
cluster_number = list(range(1, max_clusters))
for i in cluster_number:
    kmeans = KMeans(
        n_clusters=i, init="k-means++",
        n_init=10,
        tol=1e-04, random_state=42
    )
    kmeans.fit(X)
    inertia.append(kmeans.inertia_)
fig = go.Figure(data=go.Scatter(x=np.arange(1,max_clusters),y=inertia))
fig.update_layout(title="Inertia vs Cluster Number",xaxis=dict(range=[0,max_clusters],title="Cluster Number"),
                  yaxis={'title':'Inertia'},
                 annotations=[
    ])

In [95]:
#calculation of number of clusters
kneedle = KneeLocator(cluster_number, inertia, S=1.0, curve="convex", direction="decreasing")
clusters_number = kneedle.knee
if (cluster_number != int):
    kneedle = KneeLocator(cluster_number, inertia, S=0.0, curve="convex", direction="decreasing")
    clusters_number = kneedle.knee
clusters_number

/opt/anaconda3/lib/python3.9/site-packages/kneed/knee_locator.py:311: UserWarning:

No knee/elbow found



3

In [96]:
#Focusing on churn clients
x_ones.reset_index
kmeans = KMeans(
        n_clusters=clusters_number, 
        init="k-means++",
        n_init=10,
        tol=1e-04, 
        random_state=42
    )
kmeans.fit(x_ones)
clusters=pd.DataFrame(x_ones)
clusters['label']=kmeans.labels_
polar=clusters.groupby("label").mean().reset_index()
polar=pd.melt(polar,id_vars=["label"])
#Getting outliers
outliers = polar
upper = outliers['value'].quantile(0.97)
lower = outliers['value'].quantile(0.03)
outliers.groupby('variable').mean() 
outliers.loc[outliers['value'] < lower, 'outlier'] = True 
outliers.loc[outliers['value'] > upper, 'outlier'] = True 
outliers = outliers.dropna().set_index(['variable']).index
#Drop outliers
x_ones = x_ones.drop(columns=outliers).reset_index()
x_ones = x_ones.drop(columns=['index'])
x_ones



,tenure,StreamingTV,MonthlyCharges,TotalCharges,label
0,0.880672,0.238870,1.220945,0.342103,1
1,-0.463004,0.238870,-0.292844,0.285644,0
2,0.107040,0.238870,1.249194,1.539877,1
3,1.613587,0.238870,0.920182,0.587990,1
4,-1.195919,-1.071381,0.159133,-1.327393,2
...,...,...,...,...,...
1306,-1.195919,0.238870,-0.982441,0.299891,1
1307,-0.870179,-1.071381,0.052785,-1.040349,2
1308,-1.114484,1.549122,-1.492576,0.827017,1
1309,-1.236636,-1.071381,0.695855,-0.269975,2


In [97]:
#Running clustering without outliers
kmeans = KMeans(
        n_clusters=clusters_number, 
        init="k-means++",
        n_init=10,
        tol=1e-04, 
        random_state=42
    )
kmeans.fit(x_ones)
clusters=pd.DataFrame(x_ones)
clusters['label']=kmeans.labels_
#Plotting results
polar=clusters.groupby("label").mean().reset_index()
polar=pd.melt(polar,id_vars=["label"])
#plotting clusters
fig = px.line_polar(polar, r="value", theta="variable", color="label", line_close=True,height=800,width=1400)
fig.update_layout(
   title="Churn profiles",
)
fig.show()

/opt/anaconda3/lib/python3.9/site-packages/plotly/express/_core.py:271: FutureWarning:

The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.

/opt/anaconda3/lib/python3.9/site-packages/plotly/express/_core.py:271: FutureWarning:

The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.

/opt/anaconda3/lib/python3.9/site-packages/plotly/express/_core.py:271: FutureWarning:

The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.



In [98]:
#Pie plot to see clustering distribution
pie=clusters.groupby('label').size().reset_index()
pie.columns=['label','value']
px.pie(pie,values='value',names='label')